In [108]:
import pandas as pd
import numpy as np
import joblib
import os
import time
import datetime
from pytz import timezone
from datetime import timedelta
import configparser as cp
from sqlalchemy import create_engine
import mysql.connector
import yfinance as yf
from random import sample
file_path=os.getcwd()
file_path

'/home/ubuntu/projects/Stock_Price_Prediction'

# Database Connection

In [3]:
config=cp.ConfigParser()
config.read('/home/ubuntu/certi/db_login.txt')
db_config=config['ivan_db']
engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                             db_config['pwd'],
                                                                             db_config['hostname'],
                                                                             'STOCK_PRED'))

# Yahoo Finace

## 01 yfinance 
https://aroussi.com/post/python-yahoo-finance

### Testing 

In [49]:
import yfinance as yf

In [4]:
crm=yf.Ticker('CRM')
crm.history(period='max')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2004-06-23,3.750000,4.325000,3.687500,4.300000,43574400,0,0.0
2004-06-24,4.387500,4.422500,4.125000,4.190000,8887200,0,0.0
2004-06-25,4.127500,4.187500,3.947500,3.950000,6710000,0,0.0
2004-06-28,4.000000,4.052500,3.860000,4.000000,2270800,0,0.0
2004-06-29,4.000000,4.175000,3.957500,4.100000,2112000,0,0.0
...,...,...,...,...,...,...,...
2021-08-13,251.710007,253.770004,248.869995,251.559998,5985700,0,0.0
2021-08-16,250.169998,251.000000,244.470001,249.199997,4047500,0,0.0
2021-08-17,248.259995,249.199997,245.940002,246.990005,3472600,0,0.0


In [50]:
## data downloading
data=yf.download(tickers='CRM', # 'CRM AAPL MSFT'
                 #periods='max',
                 start='2021-10-06',
                 interval='1d',
                 group_by='date',
                 auto_adjust=True,
                 prepost=False, # fownlaod pre/post regular market hours data
                 threads=True,
                 proxy=None)

[*********************100%***********************]  1 of 1 completed


In [51]:
pd.DataFrame(data).reset_index()

,Date,Open,High,Low,Close,Volume
0,2021-10-06,269.670013,275.48999,267.5,275.269989,4745485


### Programmatic 

In [62]:
stock_list=[]

for i in ['CRM','AAPL','MSFT']:
    df=yf.download(tickers='{}'.format(i),
                   #periods='max',
                   start='2021-09-07',
                   interval='1d',
                   group_by='ticker',
                   auto_adjust=True,
                   prepost=False,
                   threads=True,
                   proxy=None
                  ).reset_index()
    df['stock']=i
    stock_list.append(df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [63]:
pd.concat(stock_list,axis=0)

,Date,Open,High,Low,Close,Volume,stock
0,2021-09-07,266.540009,267.899994,264.070007,265.209991,4672500,CRM
1,2021-09-08,263.619995,264.339996,261.579987,262.619995,3851500,CRM
2,2021-09-09,261.820007,263.709991,260.450012,260.739990,4126500,CRM
3,2021-09-10,261.000000,261.589996,256.500000,257.200012,5275400,CRM
4,2021-09-13,257.000000,257.690002,252.190002,254.110001,5258200,CRM
...,...,...,...,...,...,...,...
17,2021-09-30,285.709991,287.829987,281.619995,281.920013,32325800,MSFT
18,2021-10-01,282.119995,289.980011,281.290009,289.100006,30086300,MSFT
19,2021-10-04,287.399994,287.750000,280.250000,283.109985,31350700,MSFT
20,2021-10-05,284.049988,290.399994,284.049988,288.760010,24993000,MSFT


## Tickers
* https://www.nasdaq.com/market-activity/stocks/screener

### NYSE 

In [10]:
nyse=pd.read_csv(file_path+'/data/NYSE_Aug4_2021.csv')
nyse.columns=nyse.columns.str.replace(' ','').str.replace('%','')
## reformating
nyse.loc[:,'LastSale']=[float(x.replace('$',''))for x in nyse.LastSale]
nyse.loc[:,'Change']=[float(str(x).replace('%','')) for x in nyse['Change']]

print(nyse.shape)
print(nyse.Symbol.nunique())
nyse.head(5)

(3139, 11)
3139


,Symbol,Name,LastSale,NetChange,Change,MarketCap,Country,IPOYear,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,154.27,0.41,0.266,4.681219e+10,United States,1999.0,1786735,Capital Goods,Electrical Products
1,AA,Alcoa Corporation Common Stock,38.75,-1.63,-4.037,7.241064e+09,NaN,2016.0,5081059,Basic Industries,Aluminum
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,9.72,-0.02,-0.205,1.215000e+09,NaN,2021.0,164241,Finance,Business Services
3,AAIC,Arlington Asset Investment Corp Class A (new),3.78,-0.03,-0.787,1.265589e+08,United States,NaN,162347,Finance,Finance/Investors Services
4,AAIC^B,Arlington Asset Investment Corp 7.00%,25.15,0.00,0.000,NaN,United States,NaN,52,NaN,NaN


#### Reviewing - NYSE

In [11]:
nyse.dtypes

Symbol        object
Name          object
LastSale     float64
NetChange    float64
Change       float64
MarketCap    float64
Country       object
IPOYear      float64
Volume         int64
Sector        object
Industry      object
dtype: object

In [12]:
print(nyse.loc[nyse.MarketCap<=100000,'Symbol'].nunique()) # $1 million
nyse.MarketCap.describe()

128


count    2.677000e+03
mean     1.361717e+10
std      4.044338e+10
min      0.000000e+00
25%      4.256438e+08
50%      1.869335e+09
75%      8.361171e+09
max      6.157856e+11
Name: MarketCap, dtype: float64

In [13]:
## NAs
pd.DataFrame({'vals':nyse.isnull().sum()}).loc[lambda x:x.vals>0,:]

,vals
Change,1
MarketCap,462
Country,627
IPOYear,1504
Sector,752
Industry,752


#### Filtering - NYSE

In [14]:
nyse_2=nyse.loc[nyse.MarketCap>=1000000,:].reset_index(drop=True)
nyse_2.loc[:,'SE']='NYSE'
print(nyse_2.shape)
nyse_2.head(5)

(2549, 12)


,Symbol,Name,LastSale,NetChange,Change,MarketCap,Country,IPOYear,Volume,Sector,Industry,SE
0,A,Agilent Technologies Inc. Common Stock,154.27,0.41,0.266,4.681219e+10,United States,1999.0,1786735,Capital Goods,Electrical Products,NYSE
1,AA,Alcoa Corporation Common Stock,38.75,-1.63,-4.037,7.241064e+09,NaN,2016.0,5081059,Basic Industries,Aluminum,NYSE
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,9.72,-0.02,-0.205,1.215000e+09,NaN,2021.0,164241,Finance,Business Services,NYSE
3,AAIC,Arlington Asset Investment Corp Class A (new),3.78,-0.03,-0.787,1.265589e+08,United States,NaN,162347,Finance,Finance/Investors Services,NYSE
4,AAN,Aarons Holdings Company Inc. Common Stock,27.61,-1.59,-5.445,9.020895e+08,United States,2020.0,168536,Miscellaneous,Diversified Commercial Services,NYSE


In [15]:
nyse_2.to_csv(file_path+'/data/NYSE_Aug4_2021_filtered.csv',index=False)

### NASDAQ 

In [16]:
nasdaq=pd.read_csv(file_path+'/data/nasdaq_Aug4_2021.csv')
nasdaq.columns=nasdaq.columns.str.replace(' ','').str.replace('%','')
## reformating
nasdaq.loc[:,'LastSale']=[float(x.replace('$','')) for x in nasdaq.LastSale]
nasdaq.loc[:,'Change']=[float(str(x).replace('%','')) for x in nasdaq.Change]


print(nasdaq.shape)
print(nasdaq.Symbol.nunique())
nasdaq.head(5)

(4417, 11)
4417


,Symbol,Name,LastSale,NetChange,Change,MarketCap,Country,IPOYear,Volume,Sector,Industry
0,AACG,ATA Creativity Global American Depositary Shares,2.90,0.02,0.694,9.193344e+07,China,NaN,25944,Miscellaneous,Service to the Health Industry
1,AAL,American Airlines Group Inc. Common Stock,19.45,-0.50,-2.506,1.259305e+10,United States,NaN,30665670,Transportation,Air Freight/Delivery Services
2,AAME,Atlantic American Corporation Common Stock,4.20,-0.05,-1.176,8.574628e+07,United States,NaN,21993,Finance,Life Insurance
3,AAOI,Applied Optoelectronics Inc. Common Stock,7.71,0.09,1.181,2.071067e+08,United States,2013.0,563275,Technology,Semiconductors
4,AAON,AAON Inc. Common Stock,62.08,-0.90,-1.429,3.255175e+09,United States,NaN,94138,Capital Goods,Industrial Machinery/Components


#### Reviewing - Nasdaq

In [17]:
nasdaq.dtypes

Symbol        object
Name          object
LastSale     float64
NetChange    float64
Change       float64
MarketCap    float64
Country       object
IPOYear      float64
Volume         int64
Sector        object
Industry      object
dtype: object

In [18]:
print(nyse.loc[nasdaq.MarketCap<=100000,'Symbol'].nunique()) # $1 million
nasdaq.MarketCap.describe()

703


count    4.402000e+03
mean     6.340271e+09
std      7.303292e+10
min      0.000000e+00
25%      3.342577e+07
50%      2.633665e+08
75%      1.224359e+09
max      2.547722e+12
Name: MarketCap, dtype: float64

In [19]:
pd.DataFrame({'vals':nasdaq.isnull().sum()}).loc[lambda x:x.vals>0]

,vals
Change,7
MarketCap,15
Country,7
IPOYear,1626
Sector,858
Industry,859


#### Filtering - Nasdaq

In [20]:
nasdaq_2=nasdaq.loc[nasdaq.MarketCap>=1000000,:].reset_index(drop=True)
nasdaq_2.loc[:,'SE']='NASDAQ'
print(nasdaq_2.shape)
nasdaq_2.head(5)

(3442, 12)


,Symbol,Name,LastSale,NetChange,Change,MarketCap,Country,IPOYear,Volume,Sector,Industry,SE
0,AACG,ATA Creativity Global American Depositary Shares,2.90,0.02,0.694,9.193344e+07,China,NaN,25944,Miscellaneous,Service to the Health Industry,NASDAQ
1,AAL,American Airlines Group Inc. Common Stock,19.45,-0.50,-2.506,1.259305e+10,United States,NaN,30665670,Transportation,Air Freight/Delivery Services,NASDAQ
2,AAME,Atlantic American Corporation Common Stock,4.20,-0.05,-1.176,8.574628e+07,United States,NaN,21993,Finance,Life Insurance,NASDAQ
3,AAOI,Applied Optoelectronics Inc. Common Stock,7.71,0.09,1.181,2.071067e+08,United States,2013.0,563275,Technology,Semiconductors,NASDAQ
4,AAON,AAON Inc. Common Stock,62.08,-0.90,-1.429,3.255175e+09,United States,NaN,94138,Capital Goods,Industrial Machinery/Components,NASDAQ


In [21]:
## Svaing
nasdaq_2.to_csv(file_path+'/data/nasdaq_Aug4_2021_filtered.csv',index=False)

### Merge 

In [22]:
all_tickers=pd.concat([nyse_2,nasdaq_2],axis=0).reset_index(drop=True)
all_tickers.to_csv(file_path+'/data/NYSE_NASDAQ_Aug4_2021.csv',index=False)
print(all_tickers.shape)
print(all_tickers.Symbol.nunique())
all_tickers.head(5)

(5991, 12)
5991


,Symbol,Name,LastSale,NetChange,Change,MarketCap,Country,IPOYear,Volume,Sector,Industry,SE
0,A,Agilent Technologies Inc. Common Stock,154.27,0.41,0.266,4.681219e+10,United States,1999.0,1786735,Capital Goods,Electrical Products,NYSE
1,AA,Alcoa Corporation Common Stock,38.75,-1.63,-4.037,7.241064e+09,NaN,2016.0,5081059,Basic Industries,Aluminum,NYSE
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,9.72,-0.02,-0.205,1.215000e+09,NaN,2021.0,164241,Finance,Business Services,NYSE
3,AAIC,Arlington Asset Investment Corp Class A (new),3.78,-0.03,-0.787,1.265589e+08,United States,NaN,162347,Finance,Finance/Investors Services,NYSE
4,AAN,Aarons Holdings Company Inc. Common Stock,27.61,-1.59,-5.445,9.020895e+08,United States,2020.0,168536,Miscellaneous,Diversified Commercial Services,NYSE


In [23]:
all_tickers=pd.read_csv(file_path+'/data/NYSE_NASDAQ_Aug4_2021.csv')

In [24]:
## Reviewing
all_tickers.SE.value_counts()

NASDAQ    3442
NYSE      2549
Name: SE, dtype: int64

In [25]:
## saving
all_tickers.to_sql(name='NYSE_NASDAQ_TICKERS',
                   con=engine,
                   if_exists='replace',
                   index=False,
                   chunksize=1000)

## Loop 

In [26]:
stock_list=[]


for i in nasdaq_2.Symbol.tolist():
    df=yf.download(tickers='{}'.format(i),
                   periods='max',
                   interval='1d',
                   group_by='ticker',
                   auto_adjust=True,
                   prepost=False,
                   threads=True,
                   proxy=None
                  ).reset_index()
    df['stock']=i
    stock_list.append(df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


KeyboardInterrupt: 

In [ ]:
datetime.datetime.now()

In [ ]:
## All
start_time=datetime.datetime.now()
stock_list=[]
for s in ['NASDAQ','NYSE']:
    count=1
    for i in all_tickers.loc[all_tickers.SE==s,'Symbol'].tolist():
        if count%50==0:
            time.sleep(3)
            print('Wait every 50 queries: progress:{:.2f}%'.format(count/all_tickers.shape[0]*100))
        df=yf.download(tickers=i,
                       periods='max',
                       interval='1d',
                       groupby='ticker',
                       auto_adjust=True,
                       prepose=False,
                       threads=True,
                       proxy=None
                      ).reset_index()
        df['SE']=s
        df['Stock']=i
        stock_list.append(df)
        
        count+=1
        
print(datetime.datetime.now()-start_time)       

### Data Output 

In [ ]:
## 
hist_stock_price=pd.concat(stock_list)
joblib.dump(hist_stock_price,'/home/ubuntu/projects/Stock_Price_Prediction/data/stock_hist_Aug11.pkl')
print(hist_stock_price.shape)
hist_stock_price.groupby('SE').agg(stock_count=('Stock','nunique'))

In [ ]:
print()

In [81]:
file_path+'/data/NYSE_NASDAQ_Aug4_2021.csv'

'/home/ubuntu/projects/Stock_Price_Prediction/data/NYSE_NASDAQ_Aug4_2021.csv'

In [5]:
def fetch_stockprice_all(stock_exchange=['NASDAQ','NYSE'],start_vals=None,n_sample=None):
    stock_list=[]
    ## Loading tickers ##
    all_tickers=pd.read_csv(file_path+'/data/NYSE_NASDAQ_Aug4_2021.csv')
    
    ## params ##
    if start_vals==None:
        period_vals='max'
    else:
        period_vals=None
    if n_sample==None:
        all_tickers=all_tickers
    else:
        all_tickers=all_tickers.sample(n=n_sample)
    
    ## fetching data ##
    count=1
    
    for s in stock_exchange:
        for i in all_tickers.loc[all_tickers.SE==s,'Symbol'].tolist():
            if count%50==0:
                time.sleep(3)
                print('Wait every 50 queries; Progress: {:.2f}%'.format(count/all_tickers.shape[0]*100))
            
            df=yf.download(tickers=i,
                           periods=period_vals,
                           start=start_vals,
                           interval='1d',
                           groupby='ticker',
                           auto_adjust=True,
                           prepose=False,
                           threads=True,
                           proxy=None
                          ).reset_index()
            df['SE']=s
            df['Stock']=i
            stock_list.append(df)
            
            count+=1
    return pd.concat(stock_list)
    

In [21]:
df_test=fetch_stockprice_all(stock_exchange=['NASDAQ','NYSE'],start_vals='2021-09-27') ## run on Sept 23
print(df_test.shape)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BGCP: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BGFV: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BGNE: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BGRY: No data found for this date range, symbol may be delisted
Wait every 50 queries; Progress: 6.68%
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BHAT: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BHF: No data found for this date range, symbol may be delisted
[*********************100%******

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BNR: No data found for this date range, symbol may be delisted


KeyboardInterrupt: 

In [14]:
df_test.shape

(5886, 9)

In [17]:
df_test.loc[df_test.Date==pd.to_datetime('2021-09-23',format='%Y-%m-%d')]

,Date,Open,High,Low,Close,Volume,SE,Stock,Adj Close
0,2021-09-23,2.442000,2.470000,2.360000,2.360000,34490.0,NASDAQ,AACG,NaN
0,2021-09-23,20.760000,21.296499,20.650000,20.900000,51636260.0,NASDAQ,AAL,NaN
0,2021-09-23,3.790000,3.910000,3.756900,3.910000,10672.0,NASDAQ,AAME,NaN
0,2021-09-23,7.050000,7.231500,7.015000,7.160000,227299.0,NASDAQ,AAOI,NaN
0,2021-09-23,66.230003,68.150002,66.050003,68.080002,155978.0,NASDAQ,AAON,NaN
...,...,...,...,...,...,...,...,...,...
1,2021-09-23,16.040001,16.049999,15.750000,15.790000,1224354.0,NYSE,SITC,NaN
1,2021-09-23,NaN,NaN,NaN,NaN,NaN,NYSE,SRE,NaN
1,2021-09-23,NaN,NaN,NaN,NaN,NaN,NYSE,VALE,NaN
1,2021-09-23,NaN,NaN,NaN,NaN,NaN,NYSE,VICI,NaN


In [35]:
## checking initial function
df_sep23=yf.download(tickers='CRM',
                           start='2021-09-09',
                           interval='1d',
                           groupby='ticker',
                           #auto_adjust=True,
                           #prepose=False,
                           #threads=True,
                           #proxy=None
                          ).reset_index()

[*********************100%***********************]  1 of 1 completed


In [36]:
df_sep23

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-09-09,261.820007,263.709991,260.450012,260.739990,260.739990,4126500
1,2021-09-10,261.000000,261.589996,256.500000,257.200012,257.200012,5275400
2,2021-09-13,257.000000,257.690002,252.190002,254.110001,254.110001,5258200
3,2021-09-14,254.110001,255.110001,253.000000,254.369995,254.369995,4097500
4,2021-09-15,254.539993,256.750000,252.949997,256.160004,256.160004,4764600
5,2021-09-16,255.529999,261.220001,255.000000,260.359985,260.359985,5122600
6,2021-09-17,259.269989,261.700012,258.679993,260.529999,260.529999,7049400
7,2021-09-20,256.589996,259.209991,254.940002,258.220001,258.220001,5985500
8,2021-09-21,259.470001,262.549988,257.769989,257.970001,257.970001,6040600
9,2021-09-22,258.899994,260.359985,256.209991,259.170013,259.170013,4355600


In [12]:
df_test.loc[(df_test.Date==pd.to_datetime('2021-09-23',format='%Y-%m-%d'))&
            (df_test.SE=='NASDAQ'),:]

,Date,Open,High,Low,Close,Volume,SE,Stock,Adj Close
0,2021-09-13,2.850000,2.890000,2.800000,2.870000,33900.0,NASDAQ,AACG,NaN
0,2021-09-13,19.160000,19.520000,18.639999,19.309999,42886000.0,NASDAQ,AAL,NaN
0,2021-09-13,3.930000,3.990000,3.830000,3.900000,78600.0,NASDAQ,AAME,NaN
0,2021-09-13,7.440000,7.520000,7.216000,7.450000,372900.0,NASDAQ,AAOI,NaN
0,2021-09-13,65.489998,65.900002,64.230003,65.019997,295500.0,NASDAQ,AAON,NaN
...,...,...,...,...,...,...,...,...,...
0,2021-09-13,2.550000,2.590000,2.460000,2.540000,61576.0,NASDAQ,LMNL,NaN
0,2021-09-13,3.780000,3.850000,3.600700,3.850000,114344.0,NASDAQ,NBSE,NaN
0,2021-09-13,26.400000,26.400000,25.500099,25.799999,1702.0,NASDAQ,RDIB,NaN
0,2021-09-13,11550.000000,12200.000000,11450.000000,12000.000000,22382500.0,NASDAQ,SCR,NaN


In [ ]:
print(all_stock_hist.shape)

In [ ]:
# all_stock_hist.to_csv('/home/ubuntu/projects/Stock_Price_Prediction/data/stock_hist_Aug11.csv',index=False)

### Get the latest data only 

In [ ]:
hist_df=pd.read_sql("SELECT * FROM ALL_STOCK_HIST ORDER BY Date DESC LIMIT 1", con=engine)
hist_df

In [ ]:
start_dt=hist_df.loc[0,'Date']+timedelta(days=1)
start_dt

In [ ]:
fetch_stockprice_all(start_vals=start_dt)

In [ ]:
from airflow import MySQLHook

In [31]:
def update_all_stocks():
    # input new data into temp 
    # do minus
    # insert new data
    ## db connection
    config=cp.ConfigParser()
    config.read('/home/ubuntu/certi/db_login.txt')
    db_config=config['ivan_db']
    # conn -1
    engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                                   db_config['pwd'],
                                                                                   db_config['hostname'],
                                                                                 'STOCK_PRED'))
    # conn -2
    mydb=mysql.connector.connect(host=db_config['hostname'].replace(':3306',''),
                                 user=db_config['userid'],
                                 password=db_config['pwd'],
                                 database='STOCK_PRED')
    cursor=mydb.cursor()
    
    ## Get the start date:
    hist_df=pd.read_sql("SELECT * FROM ALL_STOCK_HIST ORDER BY Date DESC LIMIT 1", con=engine)
    hist_last_day=hist_df.loc[0,'Date']
    print('Latest data in the database: {:s}'.format(hist_last_day.strftime(format='%Y-%m-%d')))
    
    ## fetch the latest data
    latest_data=fetch_stockprice_all(start_vals=hist_last_day+timedelta(days=1))
    latest_data.loc[:,'REFRESH_DATE']=datetime.datetime.now()
    latest_data=latest_data.loc[:,['Date','Open','High','Low','Close','Volume','SE','Stock','REFRESH_DATE']]
    # Additional checking
    latest_data_2=latest_data.loc[latest_data.Date>hist_last_day,:].reset_index(drop=True)
    
    
    ## Load to temp table
    latest_data_2.to_sql(name='ALL_STOCK_LATEST',
                         con=engine,
                         if_exists='replace',
                         index=False,
                         chunksize=1000)
    print('ALL_STOCK_LATEST is updated with data as of {:s}'.format(latest_data.Date.max().strftime(format='%Y-%m-%d')))
    
    ## Comparing -1 
    #df_duplicate=curor.execute("""SELECT Date,Open,High,Low,Close,Volume,SE,Stock,REFRESH_DATE,COUNT(*)
    #                              FROM (SELECT *
    #                                    FROM ALL_STOCK_HIST 
    #                                    UNION ALL
    #                                    SELECT *
    #                                    FROM ALL_STOCK_LATEST)
    #                              GROUP BY Date,Open,High,Low,Close,Volume,SE,Stock,REFRESH_DATE
    #                              HAVING COUNT(*)>1""")
    #if df_duplicate.shape[0]==0:
    #    cursor.execute("""INSERT INTO ALL_STOCK_HIST
    #                      SELECT *
    #                      FROM ALL_STOCK_LATEST""")
    #    print('ALL_STOCK_HIST is updated with data as of {:s}'.format(latest_data.Date.max().strftime(format='%Y-%m-%d')))
    #    
    #else: 
    #    print('ALL_STOCK_HIST is already up-to-date')
    
    ## Comparing -2
    if hist_last_day<latest_data.Date.max():
        cursor.execute("""INSERT INTO ALL_STOCK_HIST
                          SELECT *
                          FROM ALL_STOCK_LATEST""")
        print('ALL_STOCK_HIST is updated with data as of {:s}'.format(latest_data.Date.max().strftime(format='%Y-%m-%d')))
    else:
        print('ALL_STOCK_HIST is already up-to-date, latest data: {:s}'.format(hist_last_day.strftime(format='%Y-%m-%d')))
        

In [32]:
update_all_stocks()

Latest data in the database: 2021-08-13
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 comp

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

[*********************100%***********************]  1 of 1 completed
Wait every 50 queries; Progress: 87.63%
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 comp

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [73]:
def update_all_stocks_2():
    
    utc_tz=datetime.datetime.now()
    # input new data into temp 
    # do minus
    # insert new data
    ## db connection
    config=cp.ConfigParser()
    config.read('/home/ubuntu/certi/db_login.txt')
    db_config=config['ivan_db']
    # conn -1
    engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                                   db_config['pwd'],
                                                                                   db_config['hostname'],
                                                                                 'STOCK_PRED'))
    # conn -2
    mydb=mysql.connector.connect(host=db_config['hostname'].replace(':3306',''),
                                 user=db_config['userid'],
                                 password=db_config['pwd'],
                                 database='STOCK_PRED')
    cursor=mydb.cursor()
    
    ## Get the start date:
    hist_df=pd.read_sql("SELECT * FROM ALL_STOCK_HIST ORDER BY Date DESC LIMIT 1", con=engine)
    hist_last_day=hist_df.loc[0,'Date']
    print('Latest data in the database: {:s}'.format(hist_last_day.strftime(format='%Y-%m-%d')))
    
    if utc_tz.astimezone(timezone('America/New_York')).date()>hist_last_day:
        if utc_tz.astimezone(timezone('America/New_York')).weekday()+1<=5:
            ## fetch the latest data
            latest_data=fetch_stockprice_all(start_vals=hist_last_day+timedelta(days=1))
            
            if latest_data.shape[0]>0:
                latest_data.loc[:,'REFRESH_DATE']=datetime.datetime.now()
                latest_data=latest_data.loc[:,['Date','Open','High','Low','Close','Volume','SE','Stock','REFRESH_DATE']]
        
                # additional checking
                latest_data_2=latest_data.loc[latest_data.Date>hist_last_day,:].reset_index(drop=True)
    
                ## Load to temp table 
                latest_data_2.to_sql(name='ALL_STOCK_LATEST', # ALL_STOCK_LATEST
                                     con=engine,
                                     if_exists='replace',
                                     index=False,
                                     chunksize=1000)
                print('ALL_STOCK_LATEST is updated with data as of {:s}, rows updated: {:,.0f}'.format(latest_data_2.Date.max().strftime(format='%Y-%m-%d'),
                                                                                                       latest_data_2.shape[0])
                     )
    
                cursor.execute("""INSERT INTO ALL_STOCK_HIST 
                                  SELECT *
                                  FROM ALL_STOCK_LATEST 
                                  COMMIT
                                """)
                mydb.commit()
                print('ALL_STOCK_HIST is updated with data as of {:s}'.format(latest_data_2.Date.max().strftime(format='%Y-%m-%d')))
            
            else:
                print('Stock data not available')
        
        else:
            print('Not weekday')
    
    else:
        print('ALL_STOCK_HIST is already up-to-date, latest data: {:s}'.format(hist_last_day.strftime(format='%Y-%m-%d')))
        
    return datetime.datetime.now()-utc_tz
        

In [74]:
update_all_stocks_2()

Latest data in the database: 2021-08-13
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 comp

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Wait every 50 queries; Progress: 92.64%
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 comp

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Wait every 50 queries; Progress: 98.48%
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 comp

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [76]:
hist_df=pd.read_sql("SELECT * FROM ALL_STOCK_HIST ORDER BY Date DESC LIMIT 1", con=engine)
hist_last_day=hist_df.loc[0,'Date']
hist_last_day

Timestamp('2021-08-11 00:00:00')

In [80]:
datetime.datetime(2021,9,1)

datetime.datetime(2021, 9, 1, 0, 0)

# Airflow Dag

In [94]:
from airflow import DAG
from datetime import datetime,timedelta
from airflow.operators.python import PythonOperator

## Projects
import pandas as pd
import numpy as np
import joblib
import os
import time
import datetime
from pytz import timezone
import configparser as cp
from sqlalchemy import create_engine
import mysql.connector
import yfinance as yf


## Airflow setup
yf_args={'owner':'airflow',
         'depends_on_past':False,
         'start_date':datetime(2021,9,1),
         'retries':3,
         'retry_delay':timedelta(minutes=5),
         'catchup':False # not in the documentation
        }


## db connection ##
config=cp.ConfigParser()
config.read('/home/ubuntu/certi/db_login.txt')
db_config=config['ivan_db']
# conn -1
engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                             db_config['pwd'],
                                                                             db_config['hostname'],
                                                                             'STOCK_PRED'))
# conn -2
mydb=mysql.connector.connect(host=db_config['hostname'].replace(':3306',''),
                             user=db_config['userid'],
                             password=db_config['pwd'],
                             database='STOCK_PRED')
cursor=mydb.cursor()

###############
## Functions 
###############
all_tickers=pd.read_csv('/home/ubuntu/projects/Stock_Price_Prediction/data/NYSE_NASDAQ_Aug4_2021.csv')

def fetch_stockprice_all(stock_exchange=['NASDAQ','NYSE'],start_vals=None,n_sample=None):
    stock_list=[]
    ## Loading tickers ##
    all_tickers=pd.read_csv('/home/ubuntu/projects/Stock_Price_Prediction/data/NYSE_NASDAQ_Aug4_2021.csv')
    
    ## params ##
    if start_vals==None:
        period_vals='max'
    else:
        period_vals=None
    if n_sample==None:
        all_tickers=all_tickers
    else:
        all_tickers=all_tickers.sample(n=n_sample)
    
    ## fetching data ##
    count=1
    
    for s in stock_exchange:
        for i in all_tickers.loc[all_tickers.SE==s,'Symbol'].tolist():
            if count%50==0:
                time.sleep(3)
                print('Wait every 50 queries; Progress: {:.2f}%'.format(count/all_tickers.shape[0]*100))
            
            df=yf.download(tickers=i,
                           periods=period_vals,
                           start=start_vals,
                           interval='1d',
                           groupby='ticker',
                           auto_adjust=True,
                           prepose=False,
                           threads=True,
                           proxy=None
                          ).reset_index()
            df['SE']=s
            df['Stock']=i
            stock_list.append(df)
            
            count+=1
    return pd.concat(stock_list)


###############
## Major 
###############
def update_all_stocks_2():
    
    utc_tz=datetime.datetime.now()
    # input new data into temp 
    # do minus
    # insert new data
    ## db connection
#    config=cp.ConfigParser()
#    config.read('/home/ubuntu/certi/db_login.txt')
#    db_config=config['ivan_db']
    # conn -1
#    engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
#                                                                                   db_config['pwd'],
#                                                                                   db_config['hostname'],
#                                                                                 'STOCK_PRED'))
    # conn -2
#    mydb=mysql.connector.connect(host=db_config['hostname'].replace(':3306',''),
#                                 user=db_config['userid'],
#                                 password=db_config['pwd'],
#                                 database='STOCK_PRED')
#    cursor=mydb.cursor()
    
    ## Get the start date:
    hist_df=pd.read_sql("SELECT * FROM ALL_STOCK_HIST ORDER BY Date DESC LIMIT 1", con=engine)
    hist_last_day=hist_df.loc[0,'Date']
    print('Latest data in the database: {:s}'.format(hist_last_day.strftime(format='%Y-%m-%d')))
    
    if utc_tz.astimezone(timezone('America/New_York')).date()>hist_last_day:
        if utc_tz.astimezone(timezone('America/New_York')).weekday()+1<=5:
            ## fetch the latest data
            latest_data=fetch_stockprice_all(start_vals=hist_last_day+timedelta(days=1))
            
            ## Pull the data as long as they are available
            if latest_data.shape[0]>0:  #(latest_data.shape[0]/latest_data.Date.nunique())>=5000
                latest_data.loc[:,'REFRESH_DATE']=datetime.datetime.now()
                latest_data=latest_data.loc[:,['Date','Open','High','Low','Close','Volume','SE','Stock','REFRESH_DATE']]
        
                # additional checking
                latest_data_2=latest_data.loc[latest_data.Date>hist_last_day,:].reset_index(drop=True)
    
                ## Load to temp table 
                latest_data_2.to_sql(name='ALL_STOCK_LATEST', # ALL_STOCK_LATEST
                                     con=engine,
                                     if_exists='replace',
                                     index=False,
                                     chunksize=1000)
                print('ALL_STOCK_LATEST is updated with data as of {:s}, rows updated: {:,.0f}'.format(latest_data_2.Date.max().strftime(format='%Y-%m-%d'),
                                                                                                       latest_data_2.shape[0])
                     )
    
                cursor.execute("""INSERT INTO ALL_STOCK_HIST 
                                  SELECT *
                                  FROM ALL_STOCK_LATEST 
                                  COMMIT
                                """)
                mydb.commit()
                print('ALL_STOCK_HIST is updated with data as of {:s}'.format(latest_data_2.Date.max().strftime(format='%Y-%m-%d')))
            
            else:
                print('Stock data not available; there are {0:,.0f} NA records, records per day: {1:,.0f}'.format(latest_data.Open.isnull().sum(),
                                                                                                                  latest_data.shape[0]/latest_data.Date.nunique()))
        
        else:
            print('Not weekday')
    
    else:
        print('ALL_STOCK_HIST is already up-to-date, latest data: {:s}'.format(hist_last_day.strftime(format='%Y-%m-%d')))
        

        
## Stock Attributes
def pull_stock_attri():
    
    col_attri=[]
    
    for i in all_tickers.Symbol.unique():
        stock_attri=yf.Ticker('{}'.format(i))
        result={'Stock':i,
                'Shortname':stock_attri.info['shortName'],
                'Longname':stock_attri.info['longName'],
                'Currentprice':stock_attri.info['currentPrice'],
                'Targetlowprice':stock_attri.info['targetLowPrice'],
                'Targetmeanprice':stock_attri.info['targetMeanPrice'],
                'Targetmedianprice':stock_attri.info['targetMedianPrice'],
                'Targethighprice':stock_attri.info['targetHighPrice'],
                'recommendation':stock_attri.info['recommendationKey'],
                'Date_UTC':datetime.datetime.now()
               }
        col_attri.append(result)
    
    col_attri_df=pd.DataFrame(col_attri)
    print('{} stocks are updated with data as of {}'.format(col_attri_df.Stock.nunique(),
                                                            col_attri_df.Date_UTC.max().strftime('%Y-%m-%d %H:%M')
                                                           ))
    
    ## Saving
    joblib.dump(col_attri_df,'/home/ubuntu/projects/Stock_Price_Prediction/data/stock_attris.pkl')
    
        
def load_stock_attri():
    col_attri_df=joblib.load('/home/ubuntu/projects/Stock_Price_Prediction/data/stock_attris.pkl')
    
    
    col_attri_df.to_sql(name='STOCK_ATTRIBUTES',
                        con=engine,
                        if_exists='replace',
                        index=False,
                        chunksize=1000)
        

###############
## Task 
###############
with DAG('Stock_Price_Prediction',
         default_args=yf_args,
         schedule_interval='20 0,2,22 * * *',
         cathup=False
        ) as dag:
    task_yf_base_task=PythonOperator(task_id='Yahoo_Finance_Data_Pull',
                                     python_callable=update_all_stocks_2)

    
task_yf_base_task

# use with: https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html

TypeError: 'module' object is not callable

## Manually Data Pull 

In [73]:
def fetch_stockprice_all(stock_exchange=['NASDAQ','NYSE'],start_vals=None,n_sample=None):
    stock_list=[]
    ## Loading tickers ##
    all_tickers=pd.read_csv(file_path+'/data/NYSE_NASDAQ_Aug4_2021.csv')
    
    ## params ##
    #if start_vals==None:
    #    period_vals='max'
    #else:
    #    period_vals=None
    if n_sample==None:
        all_tickers=all_tickers
    else:
        all_tickers=all_tickers.sample(n=n_sample)
    
    ## fetching data ##
    count=1
    
    for s in stock_exchange:
        for i in all_tickers.loc[all_tickers.SE==s,'Symbol'].unique():
            #if count%50==0:
            #    time.sleep(3)
            #    print('Wait every 50 queries; Progress: {:.2f}%'.format(count/all_tickers.shape[0]*100))
            print('Progress: {:.2f}%'.format(count/all_tickers.shape[0]*100))
            df=yf.download(tickers=i,
                           periods='max',
                           start=start_vals,
                           interval='1d',
                           groupby='ticker',
                           #auto_adjust=True,
                           prepose=False,
                           threads=True,
                           proxy=None
                          ).reset_index()
            df['SE']=s
            df['Stock']=i
            stock_list.append(df)
            
            count+=1
    return pd.concat(stock_list)



In [77]:
df_maunall=fetch_stockprice_all(start_vals='2021-09-07') # run on Oct 10
df_maunall.shape

Progress: 0.02%
[*********************100%***********************]  1 of 1 completed
Progress: 0.03%
[*********************100%***********************]  1 of 1 completed
Progress: 0.05%
[*********************100%***********************]  1 of 1 completed
Progress: 0.07%
[*********************100%***********************]  1 of 1 completed
Progress: 0.08%
[*********************100%***********************]  1 of 1 completed
Progress: 0.10%
[*********************100%***********************]  1 of 1 completed
Progress: 0.12%
[*********************100%***********************]  1 of 1 completed
Progress: 0.13%
[*********************100%***********************]  1 of 1 completed
Progress: 0.15%
[*********************100%***********************]  1 of 1 completed
Progress: 0.17%
[*********************100%***********************]  1 of 1 completed
Progress: 0.18%
[*********************100%***********************]  1 of 1 completed
Progress: 0.20%
[*********************100%***********************

[*********************100%***********************]  1 of 1 completed
Progress: 1.64%
[*********************100%***********************]  1 of 1 completed
Progress: 1.65%
[*********************100%***********************]  1 of 1 completed
Progress: 1.67%
[*********************100%***********************]  1 of 1 completed
Progress: 1.69%
[*********************100%***********************]  1 of 1 completed
Progress: 1.70%
[*********************100%***********************]  1 of 1 completed
Progress: 1.72%
[*********************100%***********************]  1 of 1 completed
Progress: 1.74%
[*********************100%***********************]  1 of 1 completed
Progress: 1.75%
[*********************100%***********************]  1 of 1 completed
Progress: 1.77%
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AHAC: No data found for this date range, symbol may be delisted
Progress: 1.79%
[*********************100%***********************]  1 of 1 compl

[*********************100%***********************]  1 of 1 completed
Progress: 3.20%
[*********************100%***********************]  1 of 1 completed
Progress: 3.22%
[*********************100%***********************]  1 of 1 completed
Progress: 3.24%
[*********************100%***********************]  1 of 1 completed
Progress: 3.25%
[*********************100%***********************]  1 of 1 completed
Progress: 3.27%
[*********************100%***********************]  1 of 1 completed
Progress: 3.29%
[*********************100%***********************]  1 of 1 completed
Progress: 3.30%
[*********************100%***********************]  1 of 1 completed
Progress: 3.32%
[*********************100%***********************]  1 of 1 completed
Progress: 3.34%
[*********************100%***********************]  1 of 1 completed
Progress: 3.36%
[*********************100%***********************]  1 of 1 completed
Progress: 3.37%
[*********************100%***********************]  1 of 1 comple

[*********************100%***********************]  1 of 1 completed
Progress: 4.79%
[*********************100%***********************]  1 of 1 completed
Progress: 4.81%
[*********************100%***********************]  1 of 1 completed
Progress: 4.82%
[*********************100%***********************]  1 of 1 completed
Progress: 4.84%
[*********************100%***********************]  1 of 1 completed
Progress: 4.86%
[*********************100%***********************]  1 of 1 completed
Progress: 4.87%
[*********************100%***********************]  1 of 1 completed
Progress: 4.89%
[*********************100%***********************]  1 of 1 completed
Progress: 4.91%
[*********************100%***********************]  1 of 1 completed
Progress: 4.92%
[*********************100%***********************]  1 of 1 completed
Progress: 4.94%
[*********************100%***********************]  1 of 1 completed
Progress: 4.96%
[*********************100%***********************]  1 of 1 comple

[*********************100%***********************]  1 of 1 completed
Progress: 6.39%
[*********************100%***********************]  1 of 1 completed
Progress: 6.41%
[*********************100%***********************]  1 of 1 completed
Progress: 6.43%
[*********************100%***********************]  1 of 1 completed
Progress: 6.44%
[*********************100%***********************]  1 of 1 completed
Progress: 6.46%
[*********************100%***********************]  1 of 1 completed
Progress: 6.48%
[*********************100%***********************]  1 of 1 completed
Progress: 6.49%
[*********************100%***********************]  1 of 1 completed
Progress: 6.51%
[*********************100%***********************]  1 of 1 completed
Progress: 6.53%
[*********************100%***********************]  1 of 1 completed
Progress: 6.54%
[*********************100%***********************]  1 of 1 completed
Progress: 6.56%
[*********************100%***********************]  1 of 1 comple

...

[*********************100%***********************]  1 of 1 completed
Progress: 94.98%
[*********************100%***********************]  1 of 1 completed
Progress: 94.99%
[*********************100%***********************]  1 of 1 completed
Progress: 95.01%
[*********************100%***********************]  1 of 1 completed
Progress: 95.03%
[*********************100%***********************]  1 of 1 completed
Progress: 95.04%
[*********************100%***********************]  1 of 1 completed
Progress: 95.06%
[*********************100%***********************]  1 of 1 completed
Progress: 95.08%
[*********************100%***********************]  1 of 1 completed
Progress: 95.09%
[*********************100%***********************]  1 of 1 completed
Progress: 95.11%
[*********************100%***********************]  1 of 1 completed
Progress: 95.13%
[*********************100%***********************]  1 of 1 completed
Progress: 95.14%
[*********************100%***********************]  1 

[*********************100%***********************]  1 of 1 completed
Progress: 96.56%
[*********************100%***********************]  1 of 1 completed
Progress: 96.58%
[*********************100%***********************]  1 of 1 completed
Progress: 96.59%
[*********************100%***********************]  1 of 1 completed
Progress: 96.61%
[*********************100%***********************]  1 of 1 completed
Progress: 96.63%
[*********************100%***********************]  1 of 1 completed
Progress: 96.64%
[*********************100%***********************]  1 of 1 completed
Progress: 96.66%
[*********************100%***********************]  1 of 1 completed
Progress: 96.68%
[*********************100%***********************]  1 of 1 completed
Progress: 96.70%
[*********************100%***********************]  1 of 1 completed
Progress: 96.71%
[*********************100%***********************]  1 of 1 completed
Progress: 96.73%
[*********************100%***********************]  1 

[*********************100%***********************]  1 of 1 completed
Progress: 98.15%
[*********************100%***********************]  1 of 1 completed
Progress: 98.16%
[*********************100%***********************]  1 of 1 completed
Progress: 98.18%
[*********************100%***********************]  1 of 1 completed
Progress: 98.20%
[*********************100%***********************]  1 of 1 completed
Progress: 98.21%
[*********************100%***********************]  1 of 1 completed
Progress: 98.23%
[*********************100%***********************]  1 of 1 completed
Progress: 98.25%
[*********************100%***********************]  1 of 1 completed
Progress: 98.26%
[*********************100%***********************]  1 of 1 completed
Progress: 98.28%
[*********************100%***********************]  1 of 1 completed
Progress: 98.30%
[*********************100%***********************]  1 of 1 completed
Progress: 98.31%
[*********************100%***********************]  1 

[*********************100%***********************]  1 of 1 completed
Progress: 99.73%
[*********************100%***********************]  1 of 1 completed
Progress: 99.75%
[*********************100%***********************]  1 of 1 completed
Progress: 99.77%
[*********************100%***********************]  1 of 1 completed
Progress: 99.78%
[*********************100%***********************]  1 of 1 completed
Progress: 99.80%
[*********************100%***********************]  1 of 1 completed
Progress: 99.82%
[*********************100%***********************]  1 of 1 completed
Progress: 99.83%
[*********************100%***********************]  1 of 1 completed
Progress: 99.85%
[*********************100%***********************]  1 of 1 completed
Progress: 99.87%
[*********************100%***********************]  1 of 1 completed
Progress: 99.88%
[*********************100%***********************]  1 of 1 completed
Progress: 99.90%
[*********************100%***********************]  1 

(142024, 9)

In [97]:
df_maunall.groupby(['Date']).agg(num_records=('Stock','size'))

,num_records
Date,
2021-09-07,5971
2021-09-08,5923
2021-09-09,5928
2021-09-10,5925
2021-09-13,5925
2021-09-14,5940
2021-09-15,5941
2021-09-16,5948
2021-09-17,5936


# Additional Data Points 

## Account attributes 

In [122]:
def pull_stock_attri(stocks=None,n_sample=None):
    if stocks is None:
        stocks=all_tickers.Symbol.unique()
    if (n_sample is not None) & (n_sample<len(stocks)):
        stocks=sample(set(stocks),n_sample)
        
    
    col_attri=[]
    count=0
    
    for i in stocks:
        try:
            stock_attri=yf.Ticker('{}'.format(i))
            result={'Stock':i,
                    'Shortname':stock_attri.info['shortName'],
                    'Longname':stock_attri.info['longName'],
                    'Currentprice':stock_attri.info['currentPrice'],
                    'Targetlowprice':stock_attri.info['targetLowPrice'],
                    'Targetmeanprice':stock_attri.info['targetMeanPrice'],
                    'Targetmedianprice':stock_attri.info['targetMedianPrice'],
                    'Targethighprice':stock_attri.info['targetHighPrice'],
                    'recommendation':stock_attri.info['recommendationKey'],
                    'Date_UTC':datetime.datetime.now()
                   }
            col_attri.append(result)
            count+=1
            print(count)
        
        except KeyError:
            continue
    
    col_attri_df=pd.DataFrame(col_attri)
    print('{} stocks are updated with data as of {}'.format(col_attri_df.Stock.nunique(),
                                                            col_attri_df.Date_UTC.max().strftime('%Y-%m-%d %H:%M')
                                                           ))
    
    ## Saving
    joblib.dump(col_attri_df,'/home/ubuntu/projects/Stock_Price_Prediction/data/stock_attris.pkl')
    
        
def load_stock_attri():
    col_attri_df=joblib.load('/home/ubuntu/projects/Stock_Price_Prediction/data/stock_attris.pkl')
    
    
    col_attri_df.to_sql(name='STOCK_ATTRIBUTES',
                        con=engine,
                        if_exists='replace',
                        index=False,
                        chunksize=1000)
        

### Testing 

In [123]:

pull_stock_attri(n_sample=25)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
25 stocks are updated with data as of 2021-10-13 05:00


### Review 

In [125]:
col_attri_df=joblib.load('/home/ubuntu/projects/Stock_Price_Prediction/data/stock_attris.pkl')
print(col_attri_df.shape)
print(col_attri_df.Stock.nunique())

(25, 10)
25


In [127]:
col_attri_df.head(5)

,Stock,Shortname,Longname,Currentprice,Targetlowprice,Targetmeanprice,Targetmedianprice,Targethighprice,recommendation,Date_UTC
0,MEDP,"Medpace Holdings, Inc.","Medpace Holdings, Inc.",182.74,165.0,182.00,176.0,205.0,buy,2021-10-13 04:58:25.069406
1,ADP,"Automatic Data Processing, Inc.","Automatic Data Processing, Inc.",203.84,163.0,210.81,215.0,230.0,hold,2021-10-13 04:58:29.724657
2,EEIQ,Elite Education Group Internati,Elite Education Group International Limited,4.18,10.0,10.00,10.0,10.0,none,2021-10-13 04:58:33.898166
3,INVH,Invitation Homes Inc.,Invitation Homes Inc.,39.00,40.0,44.67,45.0,49.0,buy,2021-10-13 04:58:38.131590
4,AP,Ampco-Pittsburgh Corporation,Ampco-Pittsburgh Corporation,4.44,NaN,NaN,NaN,NaN,none,2021-10-13 04:58:42.435940


In [111]:
len(all_tickers.Symbol.unique())

5991

## Account df 

In [138]:
## Recommendations - replace
def pull_stock_recommendatins(stocks=None,n_sample=None):
    if stocks is None:
        stocks=all_tickers.Symbol.unique()       
    if (n_sample is not None) & (n_sample<len(stocks)):
        stocks=sample(set(stocks),n_sample)
        
    df_col=[]
    count=0
    
    for i in stocks:
        try:
            stock_info=yf.Ticker('{}'.format(i))
            df_recommends=stock_info.recommendations.reset_index()
            df_recommends['Stock']=i
            df_col.append(df_recommends)
            count+=1
            if count%10==0:
                print('{} records downloaded, {:.1f}%'.format(count,
                                                         count/len(stocks)*100
                                                        ))
        except AttributeError:
            countine
    
    df_col=pd.concat(df_col)
    
    ## saving
    joblib.dump(df_col,'/home/ubuntu/projects/Stock_Price_Prediction/data/stock_anayst_recommendations.pkl')
    print('Stcok recommendations are updated: {:,.0f} stocks, {:,.0f} records'.format(df_col.Stock.nunique(),
                                                                                      df_col.shape(0)
                                                                                     ))
    

In [ ]:
pull_stock_recommendatins(n_sample=20)

10 records downloaded, 50.0%


## 02 yahoofinance 
* https://python-yahoofinance.readthedocs.io/en/latest/api.html

# Google Finance 

In [ ]:
from pandas_datareader import data

In [ ]:
pd.__version__